## Project Jeopardy:
This project was supposed to provide a statistical analysis of the most efficient jeopardy questions to study. The Project was a complete disaster. The instructions were just point by point do this do that and it was never explained why we were doing what we were doing. Furthermore the solutions (provieded in the instructions) were often so moronic (for loop in for loop for something that could easily have been done with vectored standard pandas operations) it hurt to execute them. I provided some of my own solutions to some of the problems, but since the project was so useless, I didn't really write many markdown cells. If you hate yourself, go on and read it. I may go back and re-do this clusterfuck at some point, some distant dark point.

In [1]:
# imports
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
jeopardy = pd.read_csv('jeopardy.csv')
pd.set_option.max_colwidth = 200

The column names have spaces in them and are capitalized. Let's fix that

In [2]:
print(jeopardy.columns)
jeopardy.columns = jeopardy.columns.str.replace('\s', '').str.lower()
print(jeopardy.columns)

Index(['Show Number', ' Air Date', ' Round', ' Category', ' Value',
       ' Question', ' Answer'],
      dtype='object')
Index(['shownumber', 'airdate', 'round', 'category', 'value', 'question',
       'answer'],
      dtype='object')


In [3]:
jeopardy.head(3)

,shownumber,airdate,round,category,value,question,answer
0,4680,2004-12-31,Jeopardy!,HISTORY,$200,"For the last 8 years of his life, Galileo was ...",Copernicus
1,4680,2004-12-31,Jeopardy!,ESPN's TOP 10 ALL-TIME ATHLETES,$200,No. 2: 1912 Olympian; football star at Carlisl...,Jim Thorpe
2,4680,2004-12-31,Jeopardy!,EVERYBODY TALKS ABOUT IT...,$200,The city of Yuma in this state has a record av...,Arizona


Let's normalize the 'answer' and 'question' columns. To do this we will make all the words lower-case and remove all the punctuation. The project guide wants us to do this with a function, which we will then apply, but it is faster, just to use .str.replace and .str.lower to do the same.

In [4]:
jeopardy['clean_question'] = jeopardy['question'].str.lower().str.replace('[^\s\w\d]', '')
jeopardy['clean_answer'] = jeopardy['answer'].str.lower().str.replace('[^\s\w\d]', '')

In [5]:
jeopardy[['clean_question', 'clean_answer']]

,clean_question,clean_answer
0,for the last 8 years of his life galileo was u...,copernicus
1,no 2 1912 olympian football star at carlisle i...,jim thorpe
2,the city of yuma in this state has a record av...,arizona
3,in 1963 live on the art linkletter show this c...,mcdonalds
4,signer of the dec of indep framer of the const...,john adams
5,in the title of an aesop fable this insect sha...,the ant
6,built in 312 bc to link rome the south of ita...,the appian way
7,no 8 30 steals for the birmingham barons 2306 ...,michael jordan
8,in the winter of 197172 a record 1122 inches o...,washington
9,this housewares store was named for the packag...,crate barrel


Now let's normalize the 'values' column, to be of the int type. NaN values will be turned to 0

In [6]:
values = jeopardy.value.str.extract('(\d+)', expand = False)
values[values.isna()] = 0
values = values.astype(int)
jeopardy['clean_values'] = values

In [7]:
jeopardy.clean_values.dtype

dtype('int64')

In [8]:
jeopardy['airdate_clean'] = pd.to_datetime(jeopardy.airdate)

In [9]:
jeopardy.airdate_clean.dtype

dtype('<M8[ns]')

We will now find out if there are questions, that include parts of their answer in the question.

In [10]:
def reoccurence(row):
    split_answer = row.clean_answer.split()
    split_question = row.clean_question.split()
    match_count = 0
    for item in ['the', 'or', '&', 'a', 'of']:
            if item in split_answer:
                split_answer.remove(item)
    for item in ['the', 'or', '&', 'a', 'of']:
            if item in split_question:
                split_question.remove(item)
    if len(split_answer) == 0:
        return 0
    
    for word in split_answer: 
        if word in split_question:
            match_count += 1
    
    return(match_count / len(split_answer))
jeopardy['answer_in_question'] = jeopardy.apply(reoccurence, axis = 1)
# jeopardy.iloc[8:10].apply(reoccurence, axis = 1)

In [11]:
jeopardy.answer_in_question.mean()*100

4.108361613493449

So we can see that about 4% of the words in the answers are mentioned in the correspondingh question.

In [12]:
jeopardy.loc[jeopardy.answer_in_question != 0, ['question', 'answer','answer_in_question']]

,question,answer,answer_in_question
14,"On June 28, 1994 the nat'l weather service beg...",the UV index,0.500000
24,This Asian political party was founded in 1885...,the Congress Party,0.500000
38,"During the 1954-1955 Sun sessions, Elvis climb...","the ""Mystery Train""",0.500000
53,"In 1961 James Brown announced ""all aboard"" for...","""Night Train""",0.500000
68,This island in the South Pacific is named for ...,Easter Island,0.500000
80,The family history you wrote for school might ...,Ellis Island,0.500000
112,He's the older son of Prince Charles and the l...,Prince William,0.500000
114,"Falcon Heene, who it turned out was safe at ho...",the balloon boy,0.500000
117,"Prime Minister Tony Blair dubbed her ""The Peop...",Princess Diana,0.500000
122,We'll look smart in these vehicles that return...,Trams (smart),0.500000


This doesn't seem very helpful when studying to win jeopardy.

We will now find out how often 

In [27]:
question_overlap = []
terms_used = set()
jeopardy = jeopardy.sort_values('airdate_clean', ascending = False)
for i, row in jeopardy.iterrows():
    match_count = 0
    split_question = row.clean_question.split()
    split_question = [q for q in split_question if len(q) > 5]

    for item in split_question:
        if item in terms_used:
            match_count += 1
    for item in split_question:
        terms_used.add(item)
    if len(split_question) > 0:
        match_count /= len(split_question)
    question_overlap.append(match_count)
jeopardy['question_overlap'] = question_overlap   
terms_list = list(terms_used)
question_overlap = []
terms_used = set()    

In [28]:
print(jeopardy.question_overlap.mean())

0.6945067290628608


This means we have proven that 70% of words longer than 5 letters are re-used in our limited dataset of jeopardy questions.

In [29]:
jeopardy.loc[jeopardy.clean_values >= 800,'high_value'] = 1
jeopardy.loc[jeopardy.clean_values < 800,'high_value'] = 0

In [30]:
jeopardy[['value', 'high_value']].head()

,value,high_value
1931,$600,0.0
1973,$2000,1.0
1971,$2000,1.0
1970,$2000,1.0
1969,$2000,1.0


The author of the project now wants us to loop over ALL the rows AGAIN and AGAIN turn the clean_question and clean_answer columns into lists. At this point I am really not sure if the author is a passable programmer. I will just make these columns part of the dataframe.

In [31]:
jeopardy['split_question'] = jeopardy.clean_question.str.split()
jeopardy['split_answer'] = jeopardy.clean_answer.str.split()

In [32]:
print(jeopardy.split_question.head())
print(jeopardy.split_answer.head())

1931    [this, singer, of, jack, diane, had, to, fight...
1973    [inopportune, or, untimely, like, the, title, ...
1971    [the, april, 2009, issue, of, science, magazin...
1970    [hefty, nickname, of, the, second, last, atomi...
1969    [to, make, this, dish, beef, is, topped, with,...
Name: split_question, dtype: object
1931    [john, mellencamp]
1973        [inconvenient]
1971       [their, genome]
1970            [fat, man]
1969    [beef, wellington]
Name: split_answer, dtype: object


In [51]:
comparison_terms = terms_list[:5]

observed_expected = []
for item in comparison_terms:
    high_count = jeopardy[jeopardy.high_value == 1].clean_question.str.contains(item).sum()
    low_count = jeopardy[jeopardy.high_value == 0].clean_question.str.contains(item).sum()
    observed_expected.append([high_count,low_count])
observed_expected

high_value_count, low_value_count = jeopardy.high_value.value_counts()[[0,1]]
print(high_value_count)
print(low_value_count)

from scipy.stats import chisquare
chi_squared = []
for entry in observed_expected:
    total = sum(entry)
    total_prob = total / jeopardy.shape[0]
    expected_high = total_prob * high_value_count
    expected_low = total_prob * low_value_count
    expected = np.array([expected_high, expected_low])
    observed = np.array(entry)
    chi_squared.append(chisquare(observed,expected))
    
chi_squared

12047
7952


[Power_divergenceResult(statistic=0.17504613678585157, pvalue=0.6756655395215962),
 Power_divergenceResult(statistic=0.6600813480534574, pvalue=0.41653122582698476),
 Power_divergenceResult(statistic=0.6600813480534574, pvalue=0.41653122582698476),
 Power_divergenceResult(statistic=0.6600813480534574, pvalue=0.41653122582698476),
 Power_divergenceResult(statistic=9.080041856857296, pvalue=0.0025841413438891787)]